# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`:

In [ ]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__


def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [ ]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__)
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [ ]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value + other.value, self.d + other.d)
        elif isinstance(other, Number):
            return Dual(self.value + other, self.d)

    def __radd__(self, other: Union["Dual", Number]) -> "Dual":
        return self + other

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value - other.value, self.d - other.d)
        elif isinstance(other, Number):
            return Dual(self.value - other, self.d)

    def __rsub__(self, other: Union["Dual", Number]) -> "Dual":
        return -self + other

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value * other.value, self.value * other.d + self.d * other.value)
        elif isinstance(other, Number):
            return Dual(self.value * other, self.d * other)

    def __rmul__(self, other: Union["Dual", Number]) -> "Dual":
        return self * other

    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        if isinstance(other, Dual):
            if other.value == 0:
                raise ValueError("Division by zero is not allowed")
            return Dual(self.value / other.value, (self.d * other.value - self.value * other.d) / (other.value ** 2))
        elif isinstance(other, Number):
            if other == 0:
                raise ValueError("Division by zero is not allowed")
            return Dual(self.value / other, self.d / other)

    def __rtruediv__(self, other: Union["Dual", Number]) -> "Dual":
        return self.__class__(1, 0) / self + other

    def __pow__(self, power: Union["Dual", Number]) -> "Dual":
        if isinstance(power, Dual):
            if self.value == 0:
                raise ValueError("Zero raised to a variable power is undefined")
            return Dual(self.value ** power.value, (power.value * self.value ** (power.value - 1)) * self.d
                        + (self.value ** power.value) * power.d)
        elif isinstance(power, Number):
            if self.value == 0 and power < 0:
                raise ValueError("Zero raised to a negative power is undefined")
            return Dual(self.value ** power, power * (self.value ** (power - 1)) * self.d)

    def __neg__(self) -> "Dual":
        return Dual(-self.value, -self.d)

def diff(func: Callable[[Dual], Dual]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d

# Тесты для проверки корректности

def test_unary_operation():
    x = 2.0
    dual_x = Dual(x, 1.0)
    result = -dual_x
    assert result.value == -x
    assert result.d == -1.0

def test_division_operation():
    x = 3.0
    y = 2.0
    dual_x = Dual(x, 1.0)
    dual_y = Dual(y, 0.0)
    result = dual_x / dual_y
    assert result.value == x / y
    assert result.d == 1 / y

def test_power_operation():
    x = 2.0
    power = 3.0
    dual_x = Dual(x, 1.0)
    dual_power = Dual(power, 0.0)
    result = dual_x ** dual_power
    assert result.value == x ** power
    assert result.d == power * x ** (power - 1)

test_unary_operation()
test_division_operation()
test_power_operation()

def f(x: Dual) -> Dual:
    return 5*x ** 2 + 2 * x + 2

f_diff = diff(f)

result = f_diff(2.0)
print(result)

# Тесты работают успешно, не вывело сообщение об ошибке

22.0


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [ ]:
import math
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value + other.value, self.d + other.d)
        elif isinstance(other, Number):
            return Dual(self.value + other, self.d)

    def __radd__(self, other: Union["Dual", Number]) -> "Dual":
        return self + other

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value - other.value, self.d - other.d)
        elif isinstance(other, Number):
            return Dual(self.value - other, self.d)

    def __rsub__(self, other: Union["Dual", Number]) -> "Dual":
        return -self + other

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
        if isinstance(other, Dual):
            return Dual(self.value * other.value, self.value * other.d + self.d * other.value)
        elif isinstance(other, Number):
            return Dual(self.value * other, self.d * other)

    def __rmul__(self, other: Union["Dual", Number]) -> "Dual":
        return self * other

    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        if isinstance(other, Dual):
            if other.value == 0:
                raise ValueError("Division by zero is not allowed")
            return Dual(self.value / other.value, (self.d * other.value - self.value * other.d) / (other.value ** 2))
        elif isinstance(other, Number):
            if other == 0:
                raise ValueError("Division by zero is not allowed")
            return Dual(self.value / other, self.d / other)

    def __rtruediv__(self, other: Union["Dual", Number]) -> "Dual":
        return self.__class__(1, 0) / self + other

    def __pow__(self, power: Union["Dual", Number]) -> "Dual":
        if isinstance(power, Dual):
            if self.value == 0:
                raise ValueError("Zero raised to a variable power is undefined")
            return Dual(self.value ** power.value, (power.value * self.value ** (power.value - 1)) * self.d
                        + (self.value ** power.value) * power.d)
        elif isinstance(power, Number):
            if self.value == 0 and power < 0:
                raise ValueError("Zero raised to a negative power is undefined")
            return Dual(self.value ** power, power * (self.value ** (power - 1)) * self.d)

    def __neg__(self) -> "Dual":
        return Dual(-self.value, -self.d)

    def exp(self) -> "Dual":
        return Dual(math.exp(self.value), self.d * math.exp(self.value))

    def cos(self) -> "Dual":
        return Dual(math.cos(self.value), -self.d * math.sin(self.value))

    def sin(self) -> "Dual":
        return Dual(math.sin(self.value), self.d * math.cos(self.value))

    def log(self) -> "Dual":
        if self.value <= 0:
            raise ValueError("log is undefined for non-positive values")
        return Dual(math.log(self.value), self.d / self.value)

def diff(func: Callable[[Dual], Dual]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d

# Тесты для проверки корректности

def test_exp_function():
    x = 2.0
    dual_x = Dual(x, 1.0)
    result = dual_x.exp()
    assert result.value == math.exp(x)
    assert result.d == math.exp(x)

def test_cos_function():
    x = math.pi / 3  # 60 degrees
    dual_x = Dual(x, 1.0)
    result = dual_x.cos()
    assert math.isclose(result.value, math.cos(x), rel_tol=1e-9)
    assert math.isclose(result.d, -math.sin(x), rel_tol=1e-9)

def test_sin_function():
    x = math.pi / 6  # 30 degrees
    dual_x = Dual(x, 1.0)
    result = dual_x.sin()
    assert math.isclose(result.value, math.sin(x), rel_tol=1e-9)
    assert math.isclose(result.d, math.cos(x), rel_tol=1e-9)

def test_log_function():
    x = 2.0
    dual_x = Dual(x, 1.0)
    result = dual_x.log()
    assert math.isclose(result.value, math.log(x), rel_tol=1e-9)
    assert math.isclose(result.d, 1 / x, rel_tol=1e-9)

test_exp_function()
test_cos_function()
test_sin_function()
test_log_function()


## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [ ]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

In [ ]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

x_value = 2.0
dual_x = Dual(x_value, 1.0)

# Используем наш метод дифференцирования
f_diff_dual = diff(f)
result_dual = f_diff_dual(x_value)

# Используем численное дифференцирование из SciPy
result_numerical = derivative(f, x_value)

# Сравниваем результаты
print("Результат с использованием нашего метода дифференцирования:", result_dual)
print("Результат с использованием численного дифференцирования из SciPy:", result_numerical)

# Проверяем, насколько близки результаты
tolerance = 1e-6
if abs(result_dual - result_numerical) < tolerance:
    print("Результаты близки или равны, код работает корректно.")
else:
    print("Результаты различаются, код требует дополнительной проверки.")


Результат с использованием нашего метода дифференцирования: 22.0
Результат с использованием численного дифференцирования из SciPy: 22.0
Результаты близки или равны, код работает корректно.


## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках

Генерацию случайных функций можно осуществить, например, двумя путями.
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [ ]:
import random
import sympy as sp
import ast

# Генерация случайных функций в текстовом виде
def generate_random_function():
    a, b = random.uniform(-10, 10), random.uniform(-10, 10)
    return f"lambda x: {a} * x + {b}"

# Генерация случайных функций с использованием ast
def generate_random_function_ast():
    a, b = random.uniform(-10, 10), random.uniform(-10, 10)
    expr = ast.Expression(
        body=ast.Lambda(
            args=ast.arguments(
                args=[
                    ast.arg(arg='x')
                ],
                posonlyargs=[],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]
            ),
            body=ast.BinOp(
                left=ast.BinOp(
                    left=ast.Constant(value=a),
                    op=ast.Mult(),
                    right=ast.Name(id='x', ctx=ast.Load())
                ),
                op=ast.Add(),
                right=ast.Constant(value=b)
            )
        )
    )
    ast.fix_missing_locations(expr)
    func = eval(compile(expr, filename="", mode="eval"))
    return func

# Символьное дифференцирование
def symbolic_differentiation(func):
    x = sp.symbols('x')
    derivative = sp.diff(func(x), x)
    return sp.lambdify(x, derivative, 'numpy')

# Численное дифференцирование
def numerical_differentiation(func, x, h=1e-5):
    return (func(x + h) - func(x)) / h

# Тестирование и сравнение результатов
def test_random_functions(num_functions, test_points):
    for _ in range(num_functions):
        func_str = generate_random_function()
        func = eval(func_str)
        func_ast = generate_random_function_ast()

        symbolic_derivative = symbolic_differentiation(func)

        print(f"Original function: {func_str}")
        print(f"Original function (AST): {func_ast}")
        print("Testing at random points:")

        for x in test_points:
            symbolic_result = symbolic_derivative(x)
            numerical_result = numerical_differentiation(func, x)

            print(f"x = {x}:")
            print(f"Symbolic Derivative: {symbolic_result}")
            print(f"Numerical Derivative: {numerical_result}")

            # Сравнивайте symbolic_result и numerical_result и выведите результат сравнения
            if abs(symbolic_result - numerical_result) < 1e-6:
                print("Results match.")
            else:
                print("Results do not match.")
            print("")

if __name__ == "__main__":
    num_functions = 5  # Количество случайных функций для тестирования
    test_points = [1.0, 2.0, 3.0]  # Точки, в которых проводится тестирование

    test_random_functions(num_functions, test_points)


Original function: lambda x: -6.1101219485472 * x + -9.402642184643534
Original function (AST): <function <lambda> at 0x0000018419C18310>
Testing at random points:
x = 1.0:
Symbolic Derivative: -6.1101219485472
Numerical Derivative: -6.110121948488255
Results match.

x = 2.0:
Symbolic Derivative: -6.1101219485472
Numerical Derivative: -6.110121948665891
Results match.

x = 3.0:
Symbolic Derivative: -6.1101219485472
Numerical Derivative: -6.110121948665891
Results match.

Original function: lambda x: 2.3289012061283128 * x + -5.14494675793715
Original function (AST): <function <lambda> at 0x0000018419C1DCA0>
Testing at random points:
x = 1.0:
Symbolic Derivative: 2.32890120612831
Numerical Derivative: 2.328901206150036
Results match.

x = 2.0:
Symbolic Derivative: 2.32890120612831
Numerical Derivative: 2.328901206105627
Results match.

x = 3.0:
Symbolic Derivative: 2.32890120612831
Numerical Derivative: 2.328901206105627
Results match.

Original function: lambda x: 7.167845862602647 * x

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [ ]:
import sympy

# Создайте символьные переменные, которые представляют аргументы функции
x, y, z = sympy.symbols('x y z')

# Определите вашу функцию
def my_function(x_val, y_val, z_val):
    return x_val * y_val + z_val - 5 * y_val

# Символьно задайте функцию
symbolic_function = x * y + z - 5 * y

# Символьно дифференцируйте функцию по аргументам
symbolic_derivative_x = sympy.diff(symbolic_function, x)
symbolic_derivative_y = sympy.diff(symbolic_function, y)
symbolic_derivative_z = sympy.diff(symbolic_function, z)

# Создайте функцию для численного дифференцирования
def diff(f):
    def numerical_derivative(x_val, y_val, z_val, h=1e-5):
        derivative_x = (f.subs({x: x_val + h, y: y_val, z: z_val}) - f.subs({x: x_val, y: y_val, z: z_val})) / h
        derivative_y = (f.subs({x: x_val, y: y_val + h, z: z_val}) - f.subs({x: x_val, y: y_val, z: z_val})) / h
        derivative_z = (f.subs({x: x_val, y: y_val, z: z_val + h}) - f.subs({x: x_val, y: y_val, z: z_val})) / h
        return [float(derivative_x), float(derivative_y), float(derivative_z)]

    return numerical_derivative

# Получите функцию для численного дифференцирования
numerical_derivative = diff(symbolic_function)

# Теперь вы можете вызывать символьную и численную производные
x_val, y_val, z_val = 10, 10, 10

symbolic_result_x = symbolic_derivative_x.subs({x: x_val, y: y_val, z: z_val})
symbolic_result_y = symbolic_derivative_y.subs({x: x_val, y: y_val, z: z_val})
symbolic_result_z = symbolic_derivative_z.subs({x: x_val, y: y_val, z: z_val})

numerical_result = numerical_derivative(x_val, y_val, z_val)

print(f"Symbolic Derivative (x): {symbolic_result_x}")
print(f"Symbolic Derivative (y): {symbolic_result_y}")
print(f"Symbolic Derivative (z): {symbolic_result_z}")
print(f"Numerical Derivative: {numerical_result}")


Symbolic Derivative (x): 10
Symbolic Derivative (y): 5
Symbolic Derivative (z): 1
Numerical Derivative: [9.999999999621423, 5.000000000165983, 1.0000000003174137]
